In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import os
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
tqdm.pandas()

In [2]:
merge_df = pd.read_csv('./Prepped Data/last_addr_coor_merge_baseline_20211129.csv')
print(merge_df.shape)
merge_df.head()

(523964, 29)


,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,일반,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,일반,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,0,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,일반,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,0,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,일반,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,일반,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,0,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [3]:
main_data_df = pd.read_csv('./Prepped Data/main_data_baseline_20211116.csv')
print(main_data_df.shape)
main_data_df.head()

(5167364, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655.0,2.0,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [4]:
main_data_df['번지'].str.contains('산').sum()

2402

In [5]:
main_data_df['번지'].isna().sum()

2720

In [6]:
main_data_df['번지'] = main_data_df['번지'].fillna('입력되지않음')

In [7]:
san_df = main_data_df[main_data_df['번지'].str.contains('산')]
print(san_df.shape)
san_df.head()

(2402, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
40057,서울특별시 관악구 신림동,산28-9,28.0,9.0,신동아,전세,84.3,201102,21,13500,0,8.0,1994.0,신림로11길 151,아파트,1억4천 이하,신동아,기타
40058,서울특별시 관악구 신림동,산28-9,28.0,9.0,신동아,전세,84.3,201104,27,16000,0,4.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타
40059,서울특별시 관악구 신림동,산28-9,28.0,9.0,신동아,전세,84.3,201107,16,18000,0,14.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타
40060,서울특별시 관악구 신림동,산28-9,28.0,9.0,신동아,전세,84.3,201109,17,16500,0,4.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타
40061,서울특별시 관악구 신림동,산28-9,28.0,9.0,신동아,전세,84.3,201112,16,15000,0,12.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타


In [8]:
nan_df = main_data_df[main_data_df['번지'] == '입력되지않음']
print(nan_df.shape)
nan_df.head()

(2720, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
556671,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,50000,0,5.0,2014.0,헌릉로8길 10-12,아파트,48000~50000,힐스테이트 서초 젠트리스,힐스테이트
556672,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,42000,0,7.0,2014.0,헌릉로8길 10-12,아파트,40000~42000,힐스테이트 서초 젠트리스,힐스테이트
556673,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,114.90,201411,30,57000,0,2.0,2014.0,헌릉로8길 10-12,아파트,56000~58000,힐스테이트 서초 젠트리스,힐스테이트
556674,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,45000,0,2.0,2014.0,헌릉로8길 10-12,아파트,44000~46000,힐스테이트 서초 젠트리스,힐스테이트
556675,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201412,3,50000,0,3.0,2014.0,헌릉로8길 10-12,아파트,48000~50000,힐스테이트 서초 젠트리스,힐스테이트


In [9]:
inv_df = main_data_df[(main_data_df['시군구'] == '서울특별시 서초구 신원동')
                      &(main_data_df['건물/단지명'] == '힐스테이트 서초 젠트리스')
                      &(main_data_df['건축년도'] == 2014)
                     ]
print(inv_df.shape)
inv_df.head()

(274, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
556671,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,50000,0,5.0,2014.0,헌릉로8길 10-12,아파트,48000~50000,힐스테이트 서초 젠트리스,힐스테이트
556672,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,42000,0,7.0,2014.0,헌릉로8길 10-12,아파트,40000~42000,힐스테이트 서초 젠트리스,힐스테이트
556673,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,114.90,201411,30,57000,0,2.0,2014.0,헌릉로8길 10-12,아파트,56000~58000,힐스테이트 서초 젠트리스,힐스테이트
556674,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201411,30,45000,0,2.0,2014.0,헌릉로8길 10-12,아파트,44000~46000,힐스테이트 서초 젠트리스,힐스테이트
556675,서울특별시 서초구 신원동,입력되지않음,NaN,NaN,힐스테이트 서초 젠트리스,전세,84.95,201412,3,50000,0,3.0,2014.0,헌릉로8길 10-12,아파트,48000~50000,힐스테이트 서초 젠트리스,힐스테이트


In [10]:
inv_df['본번'].value_counts()

Series([], Name: 본번, dtype: int64)

In [11]:
nan_df['본번'].isna().sum()

274

In [12]:
nonan_df = nan_df[nan_df['본번'].isna() == False]
print(nonan_df.shape)
nonan_df.head()

(2446, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
711869,서울특별시 서초구 내곡동,입력되지않음,0.0,0.0,서초포레스타2단지,월세,49.33,201501,28,4867,36,1.0,2015.0,헌릉로8길 45,아파트,1억4천 이하,서초포레스타단지,기타
711870,서울특별시 서초구 내곡동,입력되지않음,0.0,0.0,서초포레스타2단지,월세,49.33,201501,28,4867,36,2.0,2015.0,헌릉로8길 45,아파트,1억4천 이하,서초포레스타단지,기타
711871,서울특별시 서초구 내곡동,입력되지않음,0.0,0.0,서초포레스타2단지,전세,49.33,201501,28,11302,0,5.0,2015.0,헌릉로8길 45,아파트,1억4천 이하,서초포레스타단지,기타
711872,서울특별시 서초구 내곡동,입력되지않음,0.0,0.0,서초포레스타2단지,월세,49.33,201501,28,4867,36,3.0,2015.0,헌릉로8길 45,아파트,1억4천 이하,서초포레스타단지,기타
711873,서울특별시 서초구 내곡동,입력되지않음,0.0,0.0,서초포레스타2단지,월세,49.33,201501,28,4867,36,10.0,2015.0,헌릉로8길 45,아파트,1억4천 이하,서초포레스타단지,기타


In [13]:
nonan_df['본번'].value_counts()

0.0      2428
91.0       10
175.0       6
895.0       2
Name: 본번, dtype: int64

In [14]:
main_data_df['본번'] = main_data_df['본번'].astype('Int64')
main_data_df['부번'] = main_data_df['부번'].astype('Int64')

In [15]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [16]:
main_data_df['본번'] = main_data_df['본번'].apply(str)
main_data_df['부번'] = main_data_df['부번'].apply(str)

In [17]:
main_data_df['스플릿_0'] = main_data_df['번지'].apply(lambda x: x.split('-')[0])

In [18]:
def find_intables(x):
    # x is a intable string
    
    if pd.isna(x) == True:
        return x
    
    try:
        int(x)
        return 1
    except:
        return 0

In [19]:
main_data_df['intable'] = main_data_df['스플릿_0'].apply(find_intables)

In [20]:
non_intables_df = main_data_df[main_data_df['intable'] == 0]
print(non_intables_df.shape)
non_intables_df.head()

(7232, 20)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,스플릿_0,intable
40057,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201102,21,13500,0,8.0,1994.0,신림로11길 151,아파트,1억4천 이하,신동아,기타,산28,0
40058,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201104,27,16000,0,4.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타,산28,0
40059,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201107,16,18000,0,14.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타,산28,0
40060,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201109,17,16500,0,4.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타,산28,0
40061,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201112,16,15000,0,12.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타,산28,0


In [21]:
def number_remover(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    for i in range(10):
        x = x.replace('{}'.format(i), '')
    
    x = x.replace('-','')
        
    return x

In [22]:
non_intables_df['after_filter'] = non_intables_df['번지'].progress_apply(number_remover)

100%|██████████████████████████████████████████████████████████████████████████| 7232/7232 [00:00<00:00, 278092.40it/s]
c:\users\max6296\ml\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
non_intables_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,스플릿_0,intable,after_filter
40057,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201102,21,13500,0,8.0,1994.0,신림로11길 151,아파트,1억4천 이하,신동아,기타,산28,0,산
40058,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201104,27,16000,0,4.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타,산28,0,산
40059,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201107,16,18000,0,14.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타,산28,0,산
40060,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201109,17,16500,0,4.0,1994.0,신림로11길 151,아파트,16000~18000,신동아,기타,산28,0,산
40061,서울특별시 관악구 신림동,산28-9,28,9,신동아,전세,84.3,201112,16,15000,0,12.0,1994.0,신림로11길 151,아파트,14000~16000,신동아,기타,산28,0,산


In [24]:
non_intables_df['after_filter'].value_counts().sum()

7232

In [25]:
merge_df['대장구분명'].value_counts()

일반           522005
산              1128
가지번             689
블럭지번             99
블럭지번(지구)         25
가지번(부번세분)        17
Name: 대장구분명, dtype: int64

In [26]:
merge_df = merge_df[merge_df['대장구분명'] == '일반']
print(merge_df.shape)

(522005, 29)


In [27]:
merge_df.head()

,법정동명,대장구분명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,산여부,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,일반,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,일반,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,0,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,일반,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,0,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,일반,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,일반,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,0,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [28]:
merge_df['산여부'].value_counts()

0    522005
Name: 산여부, dtype: int64

In [29]:
merge_df = merge_df.drop(columns=['대장구분명', '산여부']).reset_index(drop=True)
print(merge_df.shape)
merge_df.head()

(522005, 27)


,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [30]:
%%time
merge_df.to_csv('./Prepped Data/merge_only_normal_20211129.csv', index=False)

Wall time: 3.44 s


In [31]:
main_data_df = main_data_df.drop(non_intables_df.index).reset_index(drop=True)
print(main_data_df.shape)
main_data_df.head()

(5160132, 20)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,스플릿_0,intable
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,655,1
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,655,1
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,655,1
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,655,1
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,655,1


In [32]:
main_data_df['번지'].isna().sum()

0

In [33]:
main_data_df['번지'].str.contains('입력').sum()

0

In [34]:
main_data_df['본번'].isna().sum()

0

In [35]:
main_data_df['부번'].isna().sum()

0

In [36]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,스플릿_0,intable
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,655,1
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,655,1
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,655,1
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,655,1
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,655,1


In [37]:
main_data_df = main_data_df.drop(columns=['스플릿_0', 'intable'])
print(main_data_df.shape)
main_data_df.head()

(5160132, 18)


,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [38]:
merge_df.head()

,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [39]:
main_data_df['본번'].dtype

dtype('O')

In [40]:
main_data_df['본번'] = main_data_df['본번'].apply(int)

In [41]:
main_data_df['본번'].dtype

dtype('int64')

In [42]:
main_data_df['부번'] = main_data_df['부번'].apply(int)

In [43]:
main_data_df['부번'].dtype

dtype('int64')

In [44]:
merge_df['본번'].dtype

dtype('int64')

In [45]:
merge_df.head()

,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [46]:
main_data_df.head()

,시군구,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [47]:
main_data_df = main_data_df.rename(columns={'시군구':'법정동명'})
print(main_data_df.shape)
main_data_df.head()

(5160132, 18)


,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대


In [50]:
main_data_df['지번주소'] = main_data_df['법정동명'] + ' ' + main_data_df['번지']

In [52]:
main_data_df.head()

,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,지번주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,서울특별시 강남구 개포동 655-2
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,서울특별시 강남구 개포동 655-2
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,서울특별시 강남구 개포동 655-2


In [53]:
main_data_df.loc[0, '지번주소']

'서울특별시 강남구 개포동 655-2'

In [54]:
zero_df = main_data_df[main_data_df['부번'] == 0]
print(zero_df.shape)
zero_df.head()

(2439906, 19)


,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,지번주소
62,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,104.43,201101,4,35000,0,12.0,1984.0,개포로 307,아파트,34000~36000,개포우성,기타,서울특별시 강남구 개포동 652
63,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,133.46,201101,8,52000,0,1.0,1984.0,개포로 307,아파트,50000~52000,개포우성,기타,서울특별시 강남구 개포동 652
64,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,133.46,201101,10,55000,0,3.0,1984.0,개포로 307,아파트,54000~56000,개포우성,기타,서울특별시 강남구 개포동 652
65,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,161.00,201101,11,63000,0,6.0,1984.0,개포로 307,아파트,62000~64000,개포우성,기타,서울특별시 강남구 개포동 652
66,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,104.43,201101,13,36000,0,10.0,1984.0,개포로 307,아파트,34000~36000,개포우성,기타,서울특별시 강남구 개포동 652


In [56]:
dash_zero_df = main_data_df[main_data_df['번지'].str.contains('-0')]
print(dash_zero_df.shape)
dash_zero_df.head()

(0, 19)


,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,지번주소


In [60]:
def dashzero_remover(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    
    x = str(x)
    
    if x.endswith('-0') == True:
        return x[:-2]

In [49]:
merge_df.head()

,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055-0,1,종교시설,종교시설
1,서울특별시 강남구 개포동,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,서울특별시 강남구 개포동 1057-9,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,서울특별시 강남구 개포동 108-1,1,주택,주택
3,서울특별시 강남구 개포동,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163-0,1,업무시설,업무시설
4,서울특별시 강남구 개포동,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,서울특별시 강남구 개포동 1163-1,1,자동차관련시설,자동차관련시설


In [61]:
merge_df['지번주소'] = merge_df['지번주소'].progress_apply(dashzero_remover)

100%|█████████████████████████████████████████████████████████████████████| 522005/522005 [00:00<00:00, 1005566.30it/s]


In [63]:
merge_df.head()

,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
0,서울특별시 강남구 개포동,1055,6899,2021,1,대,3017.0,자연녹지지역,지정되지않음,주거기타,완경사,사다리형,광대로한면,2248000,2021-07-29,1055,0,서울특별시,강남구,개포동,0,960424.632847,1.941490e+06,서울특별시 강남구 개포동 1055,1,종교시설,종교시설
1,서울특별시 강남구 개포동,1057-9,6908,2021,1,대,3062.0,개발제한구역,자연녹지지역,주거기타,고지,사다리형,광대로한면,1890000,2021-07-29,1057,9,서울특별시,강남구,개포동,0,960301.686654,1.941361e+06,None,0,입력되지않음,입력되지않음
2,서울특별시 강남구 개포동,108-1,5946,2021,1,전,24.0,자연녹지지역,지정되지않음,전,완경사,부정형,맹지,706300,2021-07-29,108,1,서울특별시,강남구,개포동,0,962252.424116,1.942756e+06,None,1,주택,주택
3,서울특별시 강남구 개포동,1163,6918,2021,1,대,335.8,제3종일반주거지역,지정되지않음,상업용,평지,세로장방,광대세각,10700000,2021-07-29,1163,0,서울특별시,강남구,개포동,0,960207.540757,1.941608e+06,서울특별시 강남구 개포동 1163,1,업무시설,업무시설
4,서울특별시 강남구 개포동,1163-1,6919,2021,1,대,336.1,제3종일반주거지역,지정되지않음,상업나지,평지,세로장방,광대세각,10700000,2021-07-29,1163,1,서울특별시,강남구,개포동,0,960234.133087,1.941599e+06,None,1,자동차관련시설,자동차관련시설


In [64]:
merge_df.loc[0, '지번주소']

'서울특별시 강남구 개포동 1055'

In [66]:
main_data_df['지번주소'].nunique()

111876

In [69]:
print(main_data_df.shape)
main_data_df.head()

(5160132, 19)


,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,지번주소
0,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,5,35000,0,7.0,1988.0,언주로 103,아파트,34000~36000,개포현대,현대,서울특별시 강남구 개포동 655-2
1,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201101,18,20000,0,8.0,1988.0,언주로 103,아파트,18000~20000,개포현대,현대,서울특별시 강남구 개포동 655-2
2,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,1,24000,0,5.0,1988.0,언주로 103,아파트,22000~24000,개포현대,현대,서울특별시 강남구 개포동 655-2
3,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,11,31000,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,서울특별시 강남구 개포동 655-2
4,서울특별시 강남구 개포동,655-2,655,2,개포2차현대아파트(220),전세,77.75,201102,24,30500,0,9.0,1988.0,언주로 103,아파트,30000~32000,개포현대,현대,서울특별시 강남구 개포동 655-2


In [67]:
main_data_selected_df = main_data_df[main_data_df['지번주소'].isin(merge_df['지번주소'].unique())]
print(main_data_selected_df.shape)
main_data_selected_df.head()

(2312467, 19)


,법정동명,번지,본번,부번,건물/단지명,거래유형,전용면적(㎡),계약년월,계약일,거래/보증금(만원),월세(만원),층,건축년도,도로명,부동산유형,가격범위,건물단지명_키워드,브랜드,지번주소
62,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,104.43,201101,4,35000,0,12.0,1984.0,개포로 307,아파트,34000~36000,개포우성,기타,서울특별시 강남구 개포동 652
63,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,133.46,201101,8,52000,0,1.0,1984.0,개포로 307,아파트,50000~52000,개포우성,기타,서울특별시 강남구 개포동 652
64,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,133.46,201101,10,55000,0,3.0,1984.0,개포로 307,아파트,54000~56000,개포우성,기타,서울특별시 강남구 개포동 652
65,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,161.00,201101,11,63000,0,6.0,1984.0,개포로 307,아파트,62000~64000,개포우성,기타,서울특별시 강남구 개포동 652
66,서울특별시 강남구 개포동,652,652,0,개포우성3차,전세,104.43,201101,13,36000,0,10.0,1984.0,개포로 307,아파트,34000~36000,개포우성,기타,서울특별시 강남구 개포동 652


In [65]:
merge_selected_df = merge_df[merge_df['지번주소'].isin(main_data_df['지번주소'].unique())]
print(merge_selected_df.shape)
merge_selected_df.head()

(7002, 27)


,법정동명,지번,토지일련번호,기준년도,기준월,지목명,토지면적,용도지역명1,용도지역명2,토지이동상황,지형높이,지형형상,도로접면,공시지가,데이터기준일자,본번,부번,시도명,시군구명,읍면동명,건물군여부,지번평균_x좌표,지번평균_y좌표,지번주소,지번별_도로명주소갯수,최빈건물용도,least_frequent_usage
11,서울특별시 강남구 개포동,1164,6928,2021,1,대,330.4,제2종일반주거지역,지정되지않음,상업기타,평지,세로장방,세로한면(가),6925000,2021-07-29,1164,0,서울특별시,강남구,개포동,0,960310.672621,1.941523e+06,서울특별시 강남구 개포동 1164,1,주택,주택
34,서울특별시 강남구 개포동,1165,6960,2021,1,대,1093.5,제2종일반주거지역,지정되지않음,아파트,평지,가로장방,세로각지(가),5242000,2021-07-29,1165,0,서울특별시,강남구,개포동,0,960538.589961,1.941757e+06,서울특별시 강남구 개포동 1165,1,주택,주택
56,서울특별시 강남구 개포동,1167,6983,2021,1,대,2145.0,제2종일반주거지역,지정되지않음,아파트,평지,가로장방,소로각지,5253000,2021-07-29,1167,0,서울특별시,강남구,개포동,0,960413.108350,1.941649e+06,서울특별시 강남구 개포동 1167,1,주택,주택
128,서울특별시 강남구 개포동,1174,7057,2021,1,대,117.3,제2종일반주거지역,지정되지않음,다세대,평지,세로장방,세로각지(가),5836000,2021-07-29,1174,0,서울특별시,강남구,개포동,0,960377.989314,1.941741e+06,서울특별시 강남구 개포동 1174,1,주택,주택
165,서울특별시 강남구 개포동,1178,7124,2021,1,대,194.7,제2종일반주거지역,지정되지않음,다세대,평지,세로장방,세로각지(가),5569000,2021-07-29,1178,0,서울특별시,강남구,개포동,0,960488.205882,1.941826e+06,서울특별시 강남구 개포동 1178,1,주택,주택


In [ ]:
main_data_df.head()